In [1]:
using Pkg, Revise
Pkg.activate(ENV["JULIA_DQMC"])
include(joinpath(ENV["JULIA_DQMC"], "src/dqmc_framework.jl"))


function mc_from_inxml(inxml::AbstractString)
    Random.seed!(1234)
    if SUPPRESS_DQMC_INIT_OUTPUT
        old_stdout = stdout
        (rp, wp) = redirect_stdout()
    end

    @assert isfile(inxml)
    p = Params()
    p.output_file = "$(inxml[1:end-7]).out.h5.running"
    xml2parameters!(p, inxml)
    mc = DQMC(p)
    init!(mc)

    if SUPPRESS_DQMC_INIT_OUTPUT
        redirect_stdout(old_stdout)
        close(wp); close(rp);
    end
    return mc
end

(@isdefined SUPPRESS_DQMC_INIT_OUTPUT) || (const SUPPRESS_DQMC_INIT_OUTPUT = false)

false

# Run all tests

In [2]:
# const SUPPRESS_DQMC_INIT_OUTPUT = true
include("runtests.jl")

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_one_plus_udt`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:141
┌ Warning: Calling potentially inaccurate `inv_one_plus_udt!`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:175
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:497
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:497
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:497
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:497
┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_sum_udts`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:497


L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = FALSE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing hopping exponentials
Initializing hopping exponentials (Checkerboard)
Checkerboard - exact (abs):		8.881784197001252e-16

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1
L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = FALSE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = FALSE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBFalse
p.hoppings = "

# Run subset of tests

In [33]:
include("tests_O3_measurements.jl")

Test Summary:      |
Boson measurements | No tests
SAFE_MULT = 1
C = 3.0
SLICES = 400
L = 10
MU1 = -0.5
SWEEPS = 20000
WRITE_EVERY_NTH = 10
GLOBAL_RATE = 5
DELTA_TAU = 0.1
SEED = 7024142146484287344
WARMUP = 1000
GIT_COMMIT = ebb8da7049a31d42490d97a0556d64f43ab064c8
MU2 = 0.5
HOPPINGS = 1.0,0.5,-0.5,-1.0
GLOBAL_UPDATES = TRUE
OPDIM = 1
U = 0.0
BFIELD = FALSE
CHECKERBOARD = FALSE
R = 1.45
LAMBDA = 0

CB = CBFalse
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_10_W_10.xml"


Loading lattice with 100 sites
Initializing neighbor-tables
Initializing hopping exponentials

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 401 -1
Allocated memory for TDGF measurement.
Test Summary:        | Pass  Total
Fermion measurements |    5      5


In [2]:
mc = mc_from_inxml("parameters\\O3_generic_small_system.in.xml")

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


DQMC of O(3) model
r = 2.0, λ = 0.5, c = 3.0, u = 1.0
Beta: 1.0 (T ≈ 1.0)
Checkerboard: CBAssaad
B-field: true

In [3]:
using BenchmarkTools

In [4]:
@btime calculate_greens($mc);

  6.340 ms (73 allocations: 1.08 MiB)


In [5]:
calculate_greens(mc);
g = copy(mc.s.greens);

In [42]:
myrdiv!(dest, a, b) = copy!(dest, adjoint(adjoint(b) \ adjoint(a)))

function calculate_greens_opt(mc::AbstractDQMC)
  s = mc.s
  tmp = mc.s.tmp
  tmp2 = mc.s.tmp2
  a = mc.a
  @mytimeit a.to "calculate_greens" begin

  mul!(tmp, s.Tl, adjoint(s.Tr))
  rmul!(tmp, Diagonal(s.Dr))
  lmul!(Diagonal(s.Dl), tmp)
  @mytimeit a.to "decompose_udt!" s.U, s.T = decompose_udt!(tmp, s.D)

  mul!(tmp, s.Ul, s.U)
  s.U .= tmp
  mul!(tmp2, s.T, adjoint(s.Ur))
        
  myrdiv!(tmp, s.U', tmp2) # or tmp = s.U' / tmp2 or mul!(tmp, adjoint(s.U), inv(tmp2))
  
  tmp[diagind(tmp)] .+= s.D
  @mytimeit a.to "decompose_udt!" u, t = decompose_udt!(tmp, s.d)

  mul!(tmp, t, tmp2)
  s.T = inv(tmp)
  mul!(tmp, s.U, u)
  s.U = adjoint(tmp)
  s.d .= 1 ./ s.d

  copyto!(tmp2, s.U)
  lmul!(Diagonal(s.d), tmp2)
  mul!(s.greens, s.T, tmp2)
  end #timeit
  nothing
end

calculate_greens_opt (generic function with 1 method)

In [43]:
@btime calculate_greens_opt($mc);
gopt = mc.s.greens
compare(gopt, g)

  5.158 ms (75 allocations: 1.08 MiB)
max absdiff: 1.0e-15
mean absdiff: 9.4e-17
max reldiff: 1.3e-11
mean reldiff: 2.2e-13
effective max reldiff: 0.0e+00
effective mean reldiff: 0.0e+00


true

In [39]:
@btime calculate_greens_opt($mc);
gopt = mc.s.greens
compare(gopt, g)

  5.134 ms (75 allocations: 1.08 MiB)
max absdiff: 1.0e-15
mean absdiff: 9.4e-17
max reldiff: 1.3e-11
mean reldiff: 2.2e-13
effective max reldiff: 0.0e+00
effective mean reldiff: 0.0e+00


true

In [11]:
@btime calculate_greens_opt($mc);
gopt = mc.s.greens
compare(gopt, g)

  5.170 ms (76 allocations: 1.14 MiB)
max absdiff: 1.0e-15
mean absdiff: 9.4e-17
max reldiff: 1.3e-11
mean reldiff: 2.2e-13
effective max reldiff: 0.0e+00
effective mean reldiff: 0.0e+00


true

In [47]:
a = rand(geltype(mc), 1000, 1000);
b = rand(geltype(mc), 1000, 1000);

@btime copy!($a, $b);
@btime copyto!($a, $b);

  1.099 ms (0 allocations: 0 bytes)
  1.108 ms (0 allocations: 0 bytes)


In [49]:
@which copyto!(a,b)

copyto!(dest::Array{T,N} where N, src::Array{T,N} where N) where T in Base at array.jl:286